In [2]:
! python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-lqcvaw24/demucs_2c44793d26564670b667efbeb97e8290
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-lqcvaw24/demucs_2c44793d26564670b667efbeb97e8290
  Resolved https://github.com/facebookresearch/demucs to commit 2e79e6e7f96401c1d35a8628e1557747a9880004
  Preparing metadata (setup.py) ... done


In [5]:
import demucs.separate
demucs.separate.main(["--float32", "--two-stems", "vocals", "-n", "mdx_extra", "/home/users/gmenon/dali/DALI_v1.0/audio/wav_clips/ad887cbfa84749e5a9789f303e2f5c30.wav"])


Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/src/notebooks/separated/mdx_extra
Separating track /home/users/gmenon/dali/DALI_v1.0/audio/wav_clips/ad887cbfa84749e5a9789f303e2f5c30.wav


100%|█████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:00<00:00, 230.12seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:00<00:00, 267.10seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:00<00:00, 276.34seconds/s]
100%|█████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:00<00:00, 282.72seconds/s]


In [6]:
demucs

<module 'demucs' from '/home/users/gmenon/.local/lib/python3.8/site-packages/demucs/__init__.py'>

In [23]:
import flash
import torch
from flash.audio import SpeechRecognition, SpeechRecognitionData
from flash.core.data.utils import download_data
import pandas as pd
from dataclasses import dataclass


@dataclass
class training_args():
    TRAIN_FILE_PATH: str = "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/train_metadata_cleaned.csv"
    TEST_FILE_PATH: str = "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_metadata_cleaned.csv"
    MODEL_BACKBONE: str = "facebook/wav2vec2-base-960h"
    # MODEL_BACKBONE =  "openai/whisper-medium", #English only
    # MODEL_BACKBONE =  "openai/whisper-large-v2" #All languages
    BATCH_SIZE: int = 8
    NUM_EPOCHS: int = 2
    NUM_GPUS = torch.cuda.device_count()
    MODEL_SAVE_PATH: str = "model_artefacts/finetuned_ALT_model.pt"
    FINETUNE_STRATEGY: str = "freeze"

In [30]:
print(f"Shape of the training file is {str(pd.read_csv(training_args.TRAIN_FILE_PATH).shape)}")
print(f"Shape of the validation files is {str(pd.read_csv(training_args.TEST_FILE_PATH).shape)}")

Shape of the training file is (133328, 4)
Shape of the validation files is (33332, 3)


In [ ]:
whisper_args = training_args(MODEL_BACKBONE = "openai/whisper-medium")
wav2vec2_args = training_args()

In [31]:
datamodule = SpeechRecognitionData.from_csv(
    "file_name",
    "transcription",
    train_file = training_args.TRAIN_FILE_PATH,
    test_file = training_args.TEST_FILE_PATH,
    batch_size = training_args.BATCH_SIZE

)

/home/users/gmenon/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3460: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)
Using 'facebook/wav2vec2-base-960h' provided by Hugging Face/transformers (https://github.com/huggingface/transformers) and PyTorch/fairseq (https://github.com/pytorch/fairseq).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Mi

Training: 0it [00:00, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.32 GiB (GPU 0; 31.74 GiB total capacity; 30.00 GiB already allocated; 179.38 MiB free; 31.16 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 2. Build the task
model = SpeechRecognition(backbone=whisper_args.MODEL_BACKBONE, MODEL_SAVE_PATH = "model_artefacts/whisper_finetuned_model.pt")
#model = SpeechRecognition(backbone="facebook/wav2vec2-large-960h-lv60-self")

# 3. Create the trainer and finetune the model
trainer = flash.Trainer(accumulate_grad_batches=10, precision=16, max_epochs=whisper_args.NUM_EPOCHS, gpus=whisper_args.NUM_GPUS)
trainer.finetune(model, datamodule=datamodule, strategy=whisper_args.FINETUNE_STRATEGY)

# 5. Save the model!
trainer.save_checkpoint(whisper_args.MODEL_SAVE_PATH)


In [4]:
# 2. Build the task
model = SpeechRecognition(backbone=wav2vec2_args.MODEL_BACKBONE, MODEL_SAVE_PATH = "model_artefacts/wav2vec2_finetuned_model.pt")
#model = SpeechRecognition(backbone="facebook/wav2vec2-large-960h-lv60-self")

# 3. Create the trainer and finetune the model
trainer = flash.Trainer(accumulate_grad_batches=10, precision=16, max_epochs=whisper_args.NUM_EPOCHS, gpus=whisper_args.NUM_GPUS)
trainer.finetune(model, datamodule=datamodule, strategy=whisper_args.FINETUNE_STRATEGY)

# 5. Save the model!
trainer.save_checkpoint(whisper_args.MODEL_SAVE_PATH)


(133328, 4)

In [ ]:
# 4. Predict on audio files!
datamodule = SpeechRecognitionData.from_files(predict_files=["/home/users/gmenon/workspace/songsLyricsGenerator/test_clip.wav"], 
                                              batch_size=whisper_args.BATCH_SIZE)
predictions = trainer.predict(model, datamodule=datamodule)
print(predictions)

In [3]:
import os
import pandas as pd

In [4]:
data = []
for folder in sorted(os.listdir('/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs')):
    for file in sorted(os.listdir('/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/'+folder)):
        if file == "vocals.wav":
            data.append((folder, file))

df = pd.DataFrame(data, columns=['Folder', 'File'])


In [5]:
df["consolidated_file_path"] = "/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/" + df["Folder"] + "/" + df["File"]

In [6]:
df.iloc[:1,-1][0]

'/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/00096edf565a4dddbb1c6071d041b4bd/vocals.wav'

In [7]:
song_metadata = pd.read_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/song_metadata_cleaned.csv")
song_metadata

,Unnamed: 0,file_name,transcription
0,0,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,heute ist wieder einer
1,1,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,der verdammten tage
2,2,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,die ich kaum ertrage
3,3,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,und mich standig selber frage
4,4,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,warum mich all
...,...,...,...
250224,250816,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,ed andiamo-avanti
250225,250817,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,soli-o-in mezzo-a tanti
250226,250818,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,sempre-accelerando
250227,250819,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,sempre fino-in fondo


In [8]:
df

,Folder,File,consolidated_file_path
0,00096edf565a4dddbb1c6071d041b4bd,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...
1,00260c535f074083bc4d3f10a526cc65,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...
2,00272e1e6b1a4271978c0f22a1985ba2,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...
3,003006ec84fb4c16bbdacbff52664921,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...
4,00467066e00d47a1a8cb530ce3ededa9,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...
...,...,...,...
8070,ffb7708228db43c8af78dada74d7156c,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...
8071,ffd9d37bf639439b9dbc0f5db30667c4,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...
8072,ffdbc01dd1414781a01b01c7afb36b3d,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...
8073,ffe1ac00f7824a35a83ad2fe2ce4ff15,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...


In [9]:
import soundfile as sf
data = []
for file in df["consolidated_file_path"]:
    f = sf.SoundFile(file)
    data.append((file,f.frames / f.samplerate))
data

[('/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/00096edf565a4dddbb1c6071d041b4bd/vocals.wav',
  1.746575963718821),
 ('/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/00260c535f074083bc4d3f10a526cc65/vocals.wav',
  0.2205668934240363),
 ('/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/00272e1e6b1a4271978c0f22a1985ba2/vocals.wav',
  1.6729478458049887),
 ('/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/003006ec84fb4c16bbdacbff52664921/vocals.wav',
  1.5028798185941044),
 ('/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/00467066e00d47a1a8cb530ce3ededa9/vocals.wav',
  2.0239455782312925),
 ('/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/0051158f3057424196b0f0b2020ee0db/vocal

In [12]:
df1 = pd.DataFrame(data, columns=['consolidated_file_path', 'Length'])
df1[df1["Length"] > 1]


,consolidated_file_path,Length
0,/home/users/gmenon/workspace/songsLyricsGenera...,1.746576
2,/home/users/gmenon/workspace/songsLyricsGenera...,1.672948
3,/home/users/gmenon/workspace/songsLyricsGenera...,1.502880
4,/home/users/gmenon/workspace/songsLyricsGenera...,2.023946
5,/home/users/gmenon/workspace/songsLyricsGenera...,2.839252
...,...,...
8069,/home/users/gmenon/workspace/songsLyricsGenera...,3.922063
8070,/home/users/gmenon/workspace/songsLyricsGenera...,2.697392
8071,/home/users/gmenon/workspace/songsLyricsGenera...,6.125011
8072,/home/users/gmenon/workspace/songsLyricsGenera...,3.729070


In [13]:
new_df = df.merge(df1, on='consolidated_file_path', how='left')
new_df

,Folder,File,consolidated_file_path,Length
0,00096edf565a4dddbb1c6071d041b4bd,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.746576
1,00260c535f074083bc4d3f10a526cc65,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,0.220567
2,00272e1e6b1a4271978c0f22a1985ba2,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.672948
3,003006ec84fb4c16bbdacbff52664921,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.502880
4,00467066e00d47a1a8cb530ce3ededa9,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,2.023946
...,...,...,...,...
8070,ffb7708228db43c8af78dada74d7156c,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,2.697392
8071,ffd9d37bf639439b9dbc0f5db30667c4,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,6.125011
8072,ffdbc01dd1414781a01b01c7afb36b3d,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,3.729070
8073,ffe1ac00f7824a35a83ad2fe2ce4ff15,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,0.974943


In [14]:
song_metadata["Folder"] = song_metadata["file_name"].str.rsplit("/", n = 1).str[-1].str.split(".").str[0]

In [15]:
song_metadata["file_name"].str.rsplit("/", n = 1).str[-1].str.split(".").str[0][0]

'c8b6a56d4f1f443786989c358361d74f'

In [26]:
song_metadata_demucs_cleaned = new_df.merge(song_metadata, on = "Folder", how = "left", indicator=True)

In [27]:
song_metadata_demucs_cleaned["transcription_capitalized"] = song_metadata_demucs_cleaned["transcription"].str.upper()

In [28]:
song_metadata_demucs_cleaned[['Folder', 'consolidated_file_path', 'Length', 'transcription','transcription_capitalized']]

,Folder,consolidated_file_path,Length,transcription,transcription_capitalized
0,00096edf565a4dddbb1c6071d041b4bd,/home/users/gmenon/workspace/songsLyricsGenera...,1.746576,est-ce que c'est ca la mort,EST-CE QUE C'EST CA LA MORT
1,00260c535f074083bc4d3f10a526cc65,/home/users/gmenon/workspace/songsLyricsGenera...,0.220567,rah,RAH
2,00272e1e6b1a4271978c0f22a1985ba2,/home/users/gmenon/workspace/songsLyricsGenera...,1.672948,"auf einmal schreien alle ""yeah!""","AUF EINMAL SCHREIEN ALLE ""YEAH!"""
3,003006ec84fb4c16bbdacbff52664921,/home/users/gmenon/workspace/songsLyricsGenera...,1.502880,then i'm gone,THEN I'M GONE
4,00467066e00d47a1a8cb530ce3ededa9,/home/users/gmenon/workspace/songsLyricsGenera...,2.023946,did they sound the death march,DID THEY SOUND THE DEATH MARCH
...,...,...,...,...,...
8070,ffb7708228db43c8af78dada74d7156c,/home/users/gmenon/workspace/songsLyricsGenera...,2.697392,qui-a sognare se vorrai tornare,QUI-A SOGNARE SE VORRAI TORNARE
8071,ffd9d37bf639439b9dbc0f5db30667c4,/home/users/gmenon/workspace/songsLyricsGenera...,6.125011,in your eyes,IN YOUR EYES
8072,ffdbc01dd1414781a01b01c7afb36b3d,/home/users/gmenon/workspace/songsLyricsGenera...,3.729070,and i miss you like crazy,AND I MISS YOU LIKE CRAZY
8073,ffe1ac00f7824a35a83ad2fe2ce4ff15,/home/users/gmenon/workspace/songsLyricsGenera...,0.974943,threw our fathers,THREW OUR FATHERS


In [23]:
song_metadata_en = pd.read_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/song_metadata_en_cleaned.csv")
song_metadata_en["Folder"] = song_metadata_en["file_name"].str.rsplit("/", n = 1).str[-1].str.split(".").str[0]
song_metadata_en_demucs_cleaned = new_df.merge(song_metadata_en, on = "Folder", how = "inner", indicator=False)
song_metadata_en_demucs_cleaned[['Folder', 'consolidated_file_path', 'Length', 'transcription']]
song_metadata_en_demucs_cleaned["transcription_capitalized"] = song_metadata_en_demucs_cleaned["transcription"].str.upper()

In [24]:
song_metadata_en_demucs_cleaned

,Folder,File,consolidated_file_path,Length,file_name,transcription,language,transcription_capitalized
0,00467066e00d47a1a8cb530ce3ededa9,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,2.023946,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,did they sound the death march,en,DID THEY SOUND THE DEATH MARCH
1,0051158f3057424196b0f0b2020ee0db,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,2.839252,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,i never felt more alone,en,I NEVER FELT MORE ALONE
2,00519afba6504271b2bf95d5b2a83795,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,0.587075,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,it's tricky,en,IT'S TRICKY
3,006b1189470141e5a69415dcc065d595,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,6.344195,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,i know i could break you down but what good wo...,en,I KNOW I COULD BREAK YOU DOWN BUT WHAT GOOD WO...
4,006e29595c604b249704b049e08f5d3b,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,3.350952,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,war is not the answer,en,WAR IS NOT THE ANSWER
...,...,...,...,...,...,...,...,...
5423,ff9e57972d0e4c9d904563c079e2c613,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,3.414195,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,the things you like,en,THE THINGS YOU LIKE
5424,ffb2b64eb4ff498a87f9a9916344179e,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,3.922063,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,something warm flowing through my eyes,en,SOMETHING WARM FLOWING THROUGH MY EYES
5425,ffd9d37bf639439b9dbc0f5db30667c4,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,6.125011,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,in your eyes,en,IN YOUR EYES
5426,ffdbc01dd1414781a01b01c7afb36b3d,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,3.729070,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,and i miss you like crazy,en,AND I MISS YOU LIKE CRAZY


In [29]:
song_metadata_en_demucs_cleaned[['Folder', 'consolidated_file_path', 'Length', 'transcription','transcription_capitalized']].to_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/song_metadata_en_demucs_cleaned.csv", index= False)
song_metadata_demucs_cleaned[['Folder', 'consolidated_file_path', 'Length', 'transcription','transcription_capitalized']].to_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/song_metadata_demucs_cleaned.csv", index= False)

In [30]:
from sklearn.model_selection import train_test_split
train_song_metadata_en_demucs_cleaned, validation_song_metadata_en_demucs_cleaned = train_test_split(song_metadata_en_demucs_cleaned, test_size=0.2)
train_song_metadata_en_demucs_cleaned.to_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/train_song_metadata_en_demucs_cleaned.csv", index= False)
validation_song_metadata_en_demucs_cleaned.to_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_song_metadata_en_demucs_cleaned.csv", index= False)

In [31]:
from sklearn.model_selection import train_test_split
train_song_metadata_demucs_cleaned, validation_song_metadata_demucs_cleaned = train_test_split(song_metadata_demucs_cleaned, test_size=0.2)
train_song_metadata_demucs_cleaned.to_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/train_song_metadata_demucs_cleaned.csv", index= False)
validation_song_metadata_demucs_cleaned.to_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_song_metadata_demucs_cleaned.csv", index= False)

In [32]:
train_song_metadata_demucs_cleaned

,Folder,File,consolidated_file_path,Length,Unnamed: 0,file_name,transcription,_merge,transcription_capitalized
5685,b49e437c29b949d5bf004ffde6773c9f,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,3.882381,166427,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,and i know,both,AND I KNOW
5892,bb1f968eb7264d6eaac52eb13dfaf3c3,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,3.593583,138155,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,by the fire in your eyes,both,BY THE FIRE IN YOUR EYES
2317,4a87958f344b45bbaaeb53248c41e82c,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.983878,86296,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,waiting to feel,both,WAITING TO FEEL
3760,7823ce444921412d8c9a77800dfdd093,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.205397,118220,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,uh shala,both,UH SHALA
6492,cdb476b7859943feb65bfe5b72099475,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,0.174444,33201,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,ha!,both,HA!
...,...,...,...,...,...,...,...,...,...
4256,88471b6ba1454791a316b8caa1100b7a,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,2.585896,38482,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,for youtocry,both,FOR YOUTOCRY
5926,bc096e9c221f4d2cad246d3df40e8ab6,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.316825,213661,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,since you been gone,both,SINCE YOU BEEN GONE
6559,cfb4e441d2b34e80a2eed5ac5c544eac,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,0.341202,35911,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,faggot,both,FAGGOT
1710,35140ba4dc7c4269b7d24082e193bd53,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,2.023129,10495,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,"feel alright, and she was",both,"FEEL ALRIGHT, AND SHE WAS"


## Readying the codebase to incorporate into python workflow

In [1]:
import os
import pandas as pd
AUDIO_FOLDER_BASE_PATH = '/scratch/users/gmenon/wav_clips/separated/htdemucs/'
data = []
for folder in sorted(os.listdir(AUDIO_FOLDER_BASE_PATH)):
    for file in sorted(os.listdir(str(AUDIO_FOLDER_BASE_PATH)+folder)):
        if file == "vocals.wav":
            data.append((str(folder), file))

df = pd.DataFrame(data, columns=['Folder', 'File'])

df["consolidated_file_path"] = AUDIO_FOLDER_BASE_PATH + df["Folder"] + "/" + df["File"]
df

,Folder,File,consolidated_file_path
0,000036667fee4665bcfec13dc0746b82,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...
1,0000d0ed017e4912a70af2aebadba2f5,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...
2,0000e96239dd46ccb22f5e3dd2722880,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...
3,00011b95376d47d8a97842cdb8a02838,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...
4,0001479ebec34bdd85d97d47c2490906,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...
...,...,...,...
24354,18fc7b628ad74d968ef5bde50bcd236e,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...
24355,18fc80bfc52f4c1185a55f3cbc2c97a2,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...
24356,18fce4d9283943e6abf996b93365f37a,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...
24357,18fe10bb0b5e4a79a192cdb27370c55c,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...


In [18]:

import soundfile as sf
data = []
for file in df["consolidated_file_path"]:
    f = sf.SoundFile(file)
    data.append((file,f.frames / f.samplerate))
df1 = pd.DataFrame(data, columns=['consolidated_file_path', 'Length'])
df1[df1["Length"] > 1]


,consolidated_file_path,Length
0,/scratch/users/gmenon/wav_clips/separated/htde...,2.160000
1,/scratch/users/gmenon/wav_clips/separated/htde...,1.187506
2,/scratch/users/gmenon/wav_clips/separated/htde...,2.325079
3,/scratch/users/gmenon/wav_clips/separated/htde...,1.607506
4,/scratch/users/gmenon/wav_clips/separated/htde...,2.496893
...,...,...
24354,/scratch/users/gmenon/wav_clips/separated/htde...,5.303129
24355,/scratch/users/gmenon/wav_clips/separated/htde...,1.500000
24356,/scratch/users/gmenon/wav_clips/separated/htde...,2.108707
24357,/scratch/users/gmenon/wav_clips/separated/htde...,2.162948


In [19]:
new_df = df.merge(df1, on='consolidated_file_path', how='left')
new_df

,Folder,File,consolidated_file_path,Length
0,000036667fee4665bcfec13dc0746b82,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,2.160000
1,0000d0ed017e4912a70af2aebadba2f5,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,1.187506
2,0000e96239dd46ccb22f5e3dd2722880,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,2.325079
3,00011b95376d47d8a97842cdb8a02838,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,1.607506
4,0001479ebec34bdd85d97d47c2490906,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,2.496893
...,...,...,...,...
24354,18fc7b628ad74d968ef5bde50bcd236e,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,5.303129
24355,18fc80bfc52f4c1185a55f3cbc2c97a2,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,1.500000
24356,18fce4d9283943e6abf996b93365f37a,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,2.108707
24357,18fe10bb0b5e4a79a192cdb27370c55c,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,2.162948


In [20]:
song_metadata = pd.read_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/song_metadata_cleaned.csv")
song_metadata["Folder"] = song_metadata["file_name"].str.rsplit("/", n = 1).str[-1].str.split(".").str[0]
song_metadata["file_name"].str.rsplit("/", n = 1).str[-1].str.split(".").str[0][0]
song_metadata_demucs_cleaned = new_df.merge(song_metadata, on = "Folder", how = "left", indicator=True)
song_metadata_demucs_cleaned["transcription_capitalized"] = song_metadata_demucs_cleaned["transcription"].str.upper()
song_metadata_demucs_cleaned[['Folder', 'consolidated_file_path', 'Length', 'transcription','transcription_capitalized']]

,Folder,consolidated_file_path,Length,transcription,transcription_capitalized
0,000036667fee4665bcfec13dc0746b82,/scratch/users/gmenon/wav_clips/separated/htde...,2.160000,solo,SOLO
1,0000d0ed017e4912a70af2aebadba2f5,/scratch/users/gmenon/wav_clips/separated/htde...,1.187506,a pyramid of cans,A PYRAMID OF CANS
2,0000e96239dd46ccb22f5e3dd2722880,/scratch/users/gmenon/wav_clips/separated/htde...,2.325079,home oh oh,HOME OH OH
3,00011b95376d47d8a97842cdb8a02838,/scratch/users/gmenon/wav_clips/separated/htde...,1.607506,fur den mob der hopfenmeile:,FUR DEN MOB DER HOPFENMEILE:
4,0001479ebec34bdd85d97d47c2490906,/scratch/users/gmenon/wav_clips/separated/htde...,2.496893,jugendliebe bringt,JUGENDLIEBE BRINGT
...,...,...,...,...,...
24354,18fc7b628ad74d968ef5bde50bcd236e,/scratch/users/gmenon/wav_clips/separated/htde...,5.303129,i must've called a thousand times,I MUST'VE CALLED A THOUSAND TIMES
24355,18fc80bfc52f4c1185a55f3cbc2c97a2,/scratch/users/gmenon/wav_clips/separated/htde...,1.500000,noches que perdi,NOCHES QUE PERDI
24356,18fce4d9283943e6abf996b93365f37a,/scratch/users/gmenon/wav_clips/separated/htde...,2.108707,we know we know we know,WE KNOW WE KNOW WE KNOW
24357,18fe10bb0b5e4a79a192cdb27370c55c,/scratch/users/gmenon/wav_clips/separated/htde...,2.162948,is hold you tight,IS HOLD YOU TIGHT


In [21]:
song_metadata_en = pd.read_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/song_metadata_en_cleaned.csv")
song_metadata_en["Folder"] = song_metadata_en["file_name"].str.rsplit("/", n = 1).str[-1].str.split(".").str[0]
song_metadata_en_demucs_cleaned = new_df.merge(song_metadata_en, on = "Folder", how = "inner", indicator=False)
song_metadata_en_demucs_cleaned[['Folder', 'consolidated_file_path', 'Length', 'transcription']]
song_metadata_en_demucs_cleaned["transcription_capitalized"] = song_metadata_en_demucs_cleaned["transcription"].str.upper()

In [24]:
BASE_PATH = '/scratch/users/gmenon/'

song_metadata_en_demucs_cleaned[['Folder', 'consolidated_file_path', 'Length', 'transcription','transcription_capitalized']].to_csv(BASE_PATH + "song_metadata_en_demucs_cleaned.csv", index= False)
song_metadata_demucs_cleaned[['Folder', 'consolidated_file_path', 'Length', 'transcription','transcription_capitalized']].to_csv(BASE_PATH + "/song_metadata_demucs_cleaned.csv", index= False)

### TRAIN TEST SPLIT

In [3]:
import pandas as pd
song_metadata_en_demucs_cleaned = pd.read_csv("/scratch/users/gmenon/song_metadata_en_demucs_cleaned.csv")
print(song_metadata_en_demucs_cleaned.shape)
song_metadata_en_demucs_cleaned

(24359, 5)


,Folder,consolidated_file_path,Length,transcription,transcription_capitalized
0,000036667fee4665bcfec13dc0746b82,/scratch/users/gmenon/wav_clips/separated/htde...,2.160000,solo,SOLO
1,0000d0ed017e4912a70af2aebadba2f5,/scratch/users/gmenon/wav_clips/separated/htde...,1.187506,a pyramid of cans,A PYRAMID OF CANS
2,0000e96239dd46ccb22f5e3dd2722880,/scratch/users/gmenon/wav_clips/separated/htde...,2.325079,home oh oh,HOME OH OH
3,00011b95376d47d8a97842cdb8a02838,/scratch/users/gmenon/wav_clips/separated/htde...,1.607506,fur den mob der hopfenmeile:,FUR DEN MOB DER HOPFENMEILE:
4,0001479ebec34bdd85d97d47c2490906,/scratch/users/gmenon/wav_clips/separated/htde...,2.496893,jugendliebe bringt,JUGENDLIEBE BRINGT
...,...,...,...,...,...
24354,18fc7b628ad74d968ef5bde50bcd236e,/scratch/users/gmenon/wav_clips/separated/htde...,5.303129,i must've called a thousand times,I MUST'VE CALLED A THOUSAND TIMES
24355,18fc80bfc52f4c1185a55f3cbc2c97a2,/scratch/users/gmenon/wav_clips/separated/htde...,1.500000,noches que perdi,NOCHES QUE PERDI
24356,18fce4d9283943e6abf996b93365f37a,/scratch/users/gmenon/wav_clips/separated/htde...,2.108707,we know we know we know,WE KNOW WE KNOW WE KNOW
24357,18fe10bb0b5e4a79a192cdb27370c55c,/scratch/users/gmenon/wav_clips/separated/htde...,2.162948,is hold you tight,IS HOLD YOU TIGHT


In [7]:
BASE_PATH = '/scratch/users/gmenon/'
from sklearn.model_selection import train_test_split
train_song_metadata_en_demucs_cleaned, validation_song_metadata_en_demucs_cleaned = train_test_split(song_metadata_en_demucs_cleaned, test_size=0.05)
train_song_metadata_en_demucs_cleaned.to_csv(BASE_PATH + "train_song_metadata_en_demucs_cleaned_095.csv", index= False)
validation_song_metadata_en_demucs_cleaned.to_csv(BASE_PATH + "validation_song_metadata_en_demucs_cleaned_005.csv", index= False)

In [9]:
train_song_metadata_en_demucs_cleaned.shape, validation_song_metadata_en_demucs_cleaned.shape

((15554, 5), (819, 5))

In [9]:
BASE_PATH = '/scratch/users/gmenon/'
song_metadata_demucs_cleaned = pd.read_csv("/scratch/users/gmenon/song_metadata_demucs_cleaned.csv")
print(song_metadata_demucs_cleaned.shape)
BASE_PATH = '/scratch/users/gmenon/'
song_metadata_demucs_cleaned

(24359, 5)


,Folder,consolidated_file_path,Length,transcription,transcription_capitalized
0,000036667fee4665bcfec13dc0746b82,/scratch/users/gmenon/wav_clips/separated/htde...,2.160000,solo,SOLO
1,0000d0ed017e4912a70af2aebadba2f5,/scratch/users/gmenon/wav_clips/separated/htde...,1.187506,a pyramid of cans,A PYRAMID OF CANS
2,0000e96239dd46ccb22f5e3dd2722880,/scratch/users/gmenon/wav_clips/separated/htde...,2.325079,home oh oh,HOME OH OH
3,00011b95376d47d8a97842cdb8a02838,/scratch/users/gmenon/wav_clips/separated/htde...,1.607506,fur den mob der hopfenmeile:,FUR DEN MOB DER HOPFENMEILE:
4,0001479ebec34bdd85d97d47c2490906,/scratch/users/gmenon/wav_clips/separated/htde...,2.496893,jugendliebe bringt,JUGENDLIEBE BRINGT
...,...,...,...,...,...
24354,18fc7b628ad74d968ef5bde50bcd236e,/scratch/users/gmenon/wav_clips/separated/htde...,5.303129,i must've called a thousand times,I MUST'VE CALLED A THOUSAND TIMES
24355,18fc80bfc52f4c1185a55f3cbc2c97a2,/scratch/users/gmenon/wav_clips/separated/htde...,1.500000,noches que perdi,NOCHES QUE PERDI
24356,18fce4d9283943e6abf996b93365f37a,/scratch/users/gmenon/wav_clips/separated/htde...,2.108707,we know we know we know,WE KNOW WE KNOW WE KNOW
24357,18fe10bb0b5e4a79a192cdb27370c55c,/scratch/users/gmenon/wav_clips/separated/htde...,2.162948,is hold you tight,IS HOLD YOU TIGHT


In [26]:
from sklearn.model_selection import train_test_split
train_song_metadata_demucs_cleaned, validation_song_metadata_demucs_cleaned = train_test_split(song_metadata_demucs_cleaned, test_size=0.05)
train_song_metadata_demucs_cleaned.to_csv(BASE_PATH + "train_song_metadata_demucs_cleaned.csv", index= False)
validation_song_metadata_demucs_cleaned.to_csv(BASE_PATH + "validation_song_metadata_demucs_cleaned.csv", index= False)

In [27]:
validation_song_metadata_en_demucs_cleaned

,Folder,File,consolidated_file_path,Length,file_name,transcription,language,transcription_capitalized
16369,18fc7b628ad74d968ef5bde50bcd236e,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,5.303129,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,i must've called a thousand times,en,I MUST'VE CALLED A THOUSAND TIMES
5008,0795d5c16ee64c9fa2134df25fb41bd4,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,3.157891,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,get away baby i want to break out,en,GET AWAY BABY I WANT TO BREAK OUT
15011,16d581f5ac0b455a9b7ec3821ab430b1,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,1.402132,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,your heart a break,en,YOUR HEART A BREAK
15593,17be53d67c3c4bc6b49e8b1f980a5cba,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,4.403764,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,fade into you,en,FADE INTO YOU
14209,159119a55a0840268ac8e6946e945a7e,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,4.231519,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,please don't make a fuss,en,PLEASE DON'T MAKE A FUSS
...,...,...,...,...,...,...,...,...
7022,0aaf765217f04cdcbd60181fafb88613,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,0.838073,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,nothing is,en,NOTHING IS
4394,06b27de6860847789840368679f3ca51,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,1.361814,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,without regard,en,WITHOUT REGARD
8451,0cf2b27ff1a0462aa1f0496625afa8d5,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,2.046077,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,and i could belong,en,AND I COULD BELONG
10457,0ffc35de5718408c8c22c1b5f259fa7d,vocals.wav,/scratch/users/gmenon/wav_clips/separated/htde...,2.550136,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,forever and always,en,FOREVER AND ALWAYS


,Folder,consolidated_file_path,Length,transcription,transcription_capitalized
0,0000d0ed017e4912a70af2aebadba2f5,/scratch/users/gmenon/wav_clips/separated/htde...,1.187506,a pyramid of cans,A PYRAMID OF CANS
1,0000e96239dd46ccb22f5e3dd2722880,/scratch/users/gmenon/wav_clips/separated/htde...,2.325079,home oh oh,HOME OH OH
2,00016cbe2b794e858e9e186297125f3b,/scratch/users/gmenon/wav_clips/separated/htde...,5.225193,but nothing seemed to light my future,BUT NOTHING SEEMED TO LIGHT MY FUTURE
3,000179f7864646b0af34eb6ed42b18bf,/scratch/users/gmenon/wav_clips/separated/htde...,1.681633,you can get whatever you need,YOU CAN GET WHATEVER YOU NEED
4,0001e8c50032442184ef04823cbed0e7,/scratch/users/gmenon/wav_clips/separated/htde...,1.272698,i would swallow my doubt,I WOULD SWALLOW MY DOUBT
...,...,...,...,...,...
16368,18fc2e5b5e8646fbb4e0b88e09d14856,/scratch/users/gmenon/wav_clips/separated/htde...,2.211950,meant to be spent alone,MEANT TO BE SPENT ALONE
16369,18fc7b628ad74d968ef5bde50bcd236e,/scratch/users/gmenon/wav_clips/separated/htde...,5.303129,i must've called a thousand times,I MUST'VE CALLED A THOUSAND TIMES
16370,18fce4d9283943e6abf996b93365f37a,/scratch/users/gmenon/wav_clips/separated/htde...,2.108707,we know we know we know,WE KNOW WE KNOW WE KNOW
16371,18fe10bb0b5e4a79a192cdb27370c55c,/scratch/users/gmenon/wav_clips/separated/htde...,2.162948,is hold you tight,IS HOLD YOU TIGHT


In [10]:
#song_metadata_demucs_cleaned.head(2).to_csv(BASE_PATH + "dummy.csv", index= False)